In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
sys.path.append("..")
from all_funcs import util
from model import Generator, Discriminator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer
tf.keras.backend.set_floatx('float64')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

In [2]:
# y_true = [3.]
# y_pred = [0.]

# bce = tf.keras.losses.MeanSquaredError()
# bce(y_true, y_pred).numpy()


In [3]:
from numba import cuda
os.environ['CUDA_VISIBLE_DEVICES']="0"
print(tf.test.is_gpu_available())

True


In [4]:
df=pd.read_csv("../dataset/df_noOutliner_ana.csv",index_col=0)
df, imp_mode, imp_mean=util.FeatureArrange(df)

In [5]:
## slices to data and labels
dataset=df.iloc[:,:-1]#.astype('float32')
org_label=df.iloc[:,-1].to_numpy().reshape(-1,1)#.astype('float32')

In [6]:
sc = MinMaxScaler()
dataset = sc.fit_transform(dataset)
dataset.shape

(4778, 72)

In [7]:
## setting hyperparameter
latent_dim = 72
epochs = 1000
batch_size= 128
buffer_size = 6000
# save_interval = 50
n_critic = 5
checkpoint_dir = './training_checkpoints'


In [8]:
generator = Generator()
discriminator = Discriminator()

In [9]:
## create Cross Entropy
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [10]:
gen_opt = tf.keras.optimizers.Adam(0.0001,0.5, 0.9)
disc_opt = tf.keras.optimizers.Adam(0.0001,)

In [11]:
@tf.function
def train_discriminator(x, labels):

    noise = tf.random.normal([x.shape[0], latent_dim])

    with tf.GradientTape() as dis_tape:
        gen_data = generator(noise, labels,)
        
        dis_output = discriminator(gen_data, label_out=False)
        
        real_output, labels_real = discriminator(x, label_out=True)
        
        labels_loss = cross_entropy(labels, labels_real)#+\
#             cross_entropy(labels, labels_fake)
        
        # formula of Gradient penalty
        x_hat = util.random_weight_average(x, gen_data)
        d_hat = discriminator(x_hat, label_out=False)

        disc_loss = util.discriminator_loss(real_output, dis_output, d_hat, x_hat)+labels_loss
        
    grad_disc = dis_tape.gradient(disc_loss, discriminator.trainable_variables)
    disc_opt.apply_gradients(zip(grad_disc, discriminator.trainable_variables))

    return disc_loss


@tf.function
def train_generator(labels):
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape:
        gen_data = generator(noise, labels,)
        dis_output, judge_labels = discriminator(gen_data, label_out=True)
        
        gen_loss = util.generator_loss(dis_output)
        
        # sum all loss
        sum_loss = gen_loss+cross_entropy(labels, judge_labels)

    grad_gen = gen_tape.gradient(sum_loss, generator.trainable_variables)
    gen_opt.apply_gradients(zip(grad_gen, generator.trainable_variables))

    return gen_loss

In [12]:
# save checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=gen_opt,
                                 discriminator_optimizer=disc_opt,
                                 generator=generator,
                                 discriminator=discriminator)

In [13]:
for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0
    
    ## create batch dataset 
    training_dataset=tf.data.Dataset.from_tensor_slices((dataset, org_label))\
    .shuffle(buffer_size).batch(batch_size, drop_remainder=True)
    
    for data, labels in training_dataset:
    
        disc_loss += train_discriminator(data, labels)
    
        if disc_opt.iterations.numpy() % n_critic == 0:
            gen_loss += train_generator(labels)
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1,time.time() - start, 
                                 gen_loss / batch_size, 
                                 disc_loss / (batch_size*n_critic)))


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Time for epoch 1 is 2.6561660766601562 sec - gen_loss = 0.015524590453700745, disc_loss = 0.31044800805085004
Time for epoch 2 is 0.33602118492126465 sec - gen_loss = 0.0076252280114854805, disc_loss = 0.028738551880803036
Time for epoch 3 is 0.3280203342437744 sec - gen_loss = 0.0006663775346641472, disc_loss = 0.003223448225940917
Time for epoch 4 is 0.3200199604034424 sec - gen_loss = 0.00015725609787923666, disc_loss = 0.005081945609209512
Time for epoch 5 is 0.3280205726623535 sec - gen_loss = 0.00

Time for epoch 67 is 0.748046875 sec - gen_loss = 2.660587772396332e-51, disc_loss = -0.0016583997631446144
Time for epoch 68 is 0.7720479965209961 sec - gen_loss = 9.401064134403036e-49, disc_loss = 0.013515095381756279
Time for epoch 69 is 0.7680482864379883 sec - gen_loss = 8.992591206679882e-47, disc_loss = 0.004439077090009262
Time for epoch 70 is 0.8040502071380615 sec - gen_loss = 2.0940786974900346e-42, disc_loss = -0.0028650272492237684
Time for epoch 71 is 0.844052791595459 sec - gen_loss = 6.095928679856824e-44, disc_loss = 0.002862199668182671
Time for epoch 72 is 0.8720545768737793 sec - gen_loss = 7.776287618608867e-42, disc_loss = 0.0012852986009915979
Time for epoch 73 is 0.9680604934692383 sec - gen_loss = 4.250163374150392e-44, disc_loss = 0.0052524966831866
Time for epoch 74 is 0.9680604934692383 sec - gen_loss = 7.85915310840307e-44, disc_loss = -0.001360804385634859
Time for epoch 75 is 1.024064064025879 sec - gen_loss = 5.4216154630639975e-49, disc_loss = 0.000559

Time for epoch 139 is 0.3960249423980713 sec - gen_loss = 1.9629028807850586e-52, disc_loss = -0.00234842762619647
Time for epoch 140 is 0.38402366638183594 sec - gen_loss = 8.98121250770882e-42, disc_loss = -0.0005077473162952344
Time for epoch 141 is 0.3960249423980713 sec - gen_loss = 3.7957078684692633e-53, disc_loss = 0.0018785501893208711
Time for epoch 142 is 0.41602611541748047 sec - gen_loss = 2.5933464598070195e-47, disc_loss = -0.001479176256193665
Time for epoch 143 is 0.3960247039794922 sec - gen_loss = 7.486987592539236e-41, disc_loss = -0.0019934254777881416
Time for epoch 144 is 0.4000251293182373 sec - gen_loss = 4.6337751789730426e-49, disc_loss = -0.00580995572418067
Time for epoch 145 is 0.4000248908996582 sec - gen_loss = 1.0400178886742014e-56, disc_loss = 0.002851614749567795
Time for epoch 146 is 0.4040253162384033 sec - gen_loss = 3.3484141712124405e-46, disc_loss = -0.0020399040311968155
Time for epoch 147 is 0.41602611541748047 sec - gen_loss = 1.538230880568

Time for epoch 210 is 0.28401780128479004 sec - gen_loss = 2.0774644773743234e-37, disc_loss = -0.0005319639993109133
Time for epoch 211 is 0.3000185489654541 sec - gen_loss = 1.7622802815364925e-48, disc_loss = -0.002464170387411542
Time for epoch 212 is 0.31601953506469727 sec - gen_loss = 3.255068882254782e-67, disc_loss = 0.0006744134543769563
Time for epoch 213 is 0.3000187873840332 sec - gen_loss = 1.668638272659641e-44, disc_loss = 0.0007487745242696591
Time for epoch 214 is 0.3040192127227783 sec - gen_loss = 1.322852778790516e-63, disc_loss = 0.0007917962974267796
Time for epoch 215 is 0.3000187873840332 sec - gen_loss = 1.0852450417085522e-48, disc_loss = 0.0020857932498439652
Time for epoch 216 is 0.30801916122436523 sec - gen_loss = 3.734732323380433e-51, disc_loss = -0.001574125781268133
Time for epoch 217 is 0.3000187873840332 sec - gen_loss = 3.34884844499873e-65, disc_loss = 0.00404692262610654
Time for epoch 218 is -20.95430874824524 sec - gen_loss = 3.582879123360771e

Time for epoch 281 is 0.5120322704315186 sec - gen_loss = 2.98013042988744e-54, disc_loss = -0.0042386617796813025
Time for epoch 282 is 0.5240328311920166 sec - gen_loss = 2.975337271074106e-48, disc_loss = -0.0003397373465695283
Time for epoch 283 is 0.5320332050323486 sec - gen_loss = 5.56237363113731e-48, disc_loss = -0.0021377410459333312
Time for epoch 284 is 0.5440340042114258 sec - gen_loss = 1.0230077640758253e-43, disc_loss = -0.0004046027644039983
Time for epoch 285 is 0.5360333919525146 sec - gen_loss = 1.5029881022175968e-44, disc_loss = 0.0003112398713654844
Time for epoch 286 is 0.5240325927734375 sec - gen_loss = 2.671504756436288e-35, disc_loss = -0.0007408524576713576
Time for epoch 287 is 0.5320332050323486 sec - gen_loss = 3.2638616709048823e-68, disc_loss = -0.0005032745981624457
Time for epoch 288 is 0.572035551071167 sec - gen_loss = 1.2501039540994267e-54, disc_loss = -0.0017772637084498368
Time for epoch 289 is 0.5920369625091553 sec - gen_loss = 7.361507314720

...

Time for epoch 707 is 0.33602094650268555 sec - gen_loss = 6.085441263284169e-44, disc_loss = -0.004253592017693526
Time for epoch 708 is 0.33602094650268555 sec - gen_loss = 4.2207539385380825e-58, disc_loss = 0.0012596867234926216
Time for epoch 709 is 0.34002113342285156 sec - gen_loss = 2.1564179890349137e-48, disc_loss = -0.0017327682391098998
Time for epoch 710 is 0.3440215587615967 sec - gen_loss = 1.7277072842531844e-58, disc_loss = 0.0008089670335758754
Time for epoch 711 is 0.3440215587615967 sec - gen_loss = 1.6187161147811804e-56, disc_loss = -0.0002811806370074511
Time for epoch 712 is 0.3440215587615967 sec - gen_loss = 2.173150025680308e-74, disc_loss = -0.0025246544800728685
Time for epoch 713 is 0.3440215587615967 sec - gen_loss = 6.474923989951983e-74, disc_loss = -0.0010556437575721341
Time for epoch 714 is 0.3520219326019287 sec - gen_loss = 1.0432976953433698e-59, disc_loss = -0.0033652105300640083
Time for epoch 715 is 0.3520216941833496 sec - gen_loss = 1.0246964

Time for epoch 778 is 1.2800800800323486 sec - gen_loss = 4.168236665427097e-54, disc_loss = 0.002770854738177943
Time for epoch 779 is 1.420088768005371 sec - gen_loss = 1.9408656051758548e-33, disc_loss = 0.003220819244571986
Time for epoch 780 is 1.6321017742156982 sec - gen_loss = 1.4999047341966255e-63, disc_loss = -0.005881399562463217
Time for epoch 781 is 1.9601225852966309 sec - gen_loss = 1.1593833693544881e-57, disc_loss = -0.0022196442078965704
Time for epoch 782 is 2.6801676750183105 sec - gen_loss = 2.5337505907816936e-48, disc_loss = -0.0027144314084520385
Time for epoch 783 is 5.3123321533203125 sec - gen_loss = 8.971516295031957e-54, disc_loss = -0.0006800136257840388
Time for epoch 784 is 0.2760169506072998 sec - gen_loss = 7.596877524094048e-64, disc_loss = -0.0034205257829417225
Time for epoch 785 is 0.2720170021057129 sec - gen_loss = 5.6093879559809255e-67, disc_loss = -0.007168283961144456
Time for epoch 786 is 0.268017053604126 sec - gen_loss = 3.545880242291005

Time for epoch 849 is 0.38802433013916016 sec - gen_loss = 6.141981732158587e-60, disc_loss = -0.0012652721499050474
Time for epoch 850 is 0.41602587699890137 sec - gen_loss = 5.020662561340696e-68, disc_loss = -0.007745951914756194
Time for epoch 851 is 0.41602611541748047 sec - gen_loss = 5.170458088387606e-57, disc_loss = -0.0049644206985093595
Time for epoch 852 is 0.4200265407562256 sec - gen_loss = 5.2407985300590305e-61, disc_loss = -0.005024334663094085
Time for epoch 853 is 0.4280266761779785 sec - gen_loss = 6.220688256967743e-57, disc_loss = -0.0052612109945265675
Time for epoch 854 is 0.43202686309814453 sec - gen_loss = 4.793988410370035e-46, disc_loss = -0.0019211621101598753
Time for epoch 855 is 0.4280266761779785 sec - gen_loss = 8.902108784894672e-57, disc_loss = 0.001280245738396439
Time for epoch 856 is 0.43602705001831055 sec - gen_loss = 1.2159486776517074e-51, disc_loss = -0.0025264399152442685
Time for epoch 857 is 0.4520285129547119 sec - gen_loss = 1.447063948

Time for epoch 920 is 0.3000187873840332 sec - gen_loss = 1.95117667174023e-35, disc_loss = -0.0017315891396253336
Time for epoch 921 is 0.3000187873840332 sec - gen_loss = 6.230725788167463e-35, disc_loss = -0.0025261439544019076
Time for epoch 922 is 0.3040189743041992 sec - gen_loss = 9.581896369388077e-55, disc_loss = -0.0023916041819679068
Time for epoch 923 is 0.3040189743041992 sec - gen_loss = 1.4021021150119981e-43, disc_loss = -0.002248088090639385
Time for epoch 924 is 0.3000187873840332 sec - gen_loss = 3.145910059481424e-57, disc_loss = 0.005100058775504577
Time for epoch 925 is 0.3000190258026123 sec - gen_loss = 1.060866506127159e-30, disc_loss = 0.001458472678151466
Time for epoch 926 is 0.3040189743041992 sec - gen_loss = 1.630280092357354e-20, disc_loss = 0.0052804325925797395
Time for epoch 927 is 0.30801939964294434 sec - gen_loss = 3.7717190811134274e-52, disc_loss = -0.00023149375535721443
Time for epoch 928 is 0.31601953506469727 sec - gen_loss = 9.22658084972262

Time for epoch 991 is 0.5640354156494141 sec - gen_loss = 1.6570284260099902e-32, disc_loss = -0.004513445913320205
Time for epoch 992 is 0.5760359764099121 sec - gen_loss = 1.1775740454642496e-39, disc_loss = -0.0030988040266102914
Time for epoch 993 is 0.6120383739471436 sec - gen_loss = 4.157968981492642e-35, disc_loss = -0.00631891082572435
Time for epoch 994 is 0.6040375232696533 sec - gen_loss = 2.565380632345292e-28, disc_loss = -0.0030796462668009985
Time for epoch 995 is 0.6040375232696533 sec - gen_loss = 7.0206232563625535e-25, disc_loss = -0.0058303423997197205
Time for epoch 996 is 0.6280395984649658 sec - gen_loss = 3.45444328482861e-19, disc_loss = -5.57911836773289e-05
Time for epoch 997 is 0.6480402946472168 sec - gen_loss = 7.68223755143113e-33, disc_loss = -0.0032012117261140892
Time for epoch 998 is 0.660041332244873 sec - gen_loss = 2.1470235075256382e-23, disc_loss = -0.003804853966067642
Time for epoch 999 is 0.6720418930053711 sec - gen_loss = 4.781917732827919e

In [14]:
noise = tf.random.normal([128*2, latent_dim])
zero_label=tf.zeros((128),)
one_label =tf.ones((128),)
all_labels=tf.reshape(tf.stack([zero_label,one_label]),(-1,1))

a=generator(noise,all_labels).numpy()
# (sc.inverse_transform(a))
# for i in sc.inverse_transform(a):
#     print(np.round(i))


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [19]:
output_dataset=pd.DataFrame(np.round(sc.inverse_transform(a)),columns=[
'BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM', 'PTT1_NM',
'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI', 'age', 'NIHTotal','PPD',
'THD_ID', 'THDA_FL', 'THDH_FL', 'THDI_FL',
'THDAM_FL', 'THDV_FL', 'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL',
'THDOO_FL', 'Gender', 'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
'Hemorrhagic_infarct_ctl', 'cortical_CT', 'subcortical_CT',
'circulation_CT', 'CT_find', 'watershed_CT', 'Hemorrhagic_infarct_CT',
'CT_left', 'CT_right', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
'NIHS_10_in', 'NIHS_11_in',  
])
output_dataset

,BT_NM,HR_NM,RR_NM,HB_NM,HCT_NM,PLATELET_NM,WBC_NM,PTT1_NM,PTT2_NM,PTINR_NM,ER_NM,BUN_NM,CRE_NM,BMI,age,NIHTotal,PPD,THD_ID,THDA_FL,THDH_FL,THDI_FL,THDAM_FL,THDV_FL,THDE_FL,THDM_FL,THDR_FL,THDP_FL,THDOO_FL,Gender,cortical_ACA_ctr,cortical_MCA_ctr,subcortical_ACA_ctr,subcortical_MCA_ctr,PCA_cortex_ctr,thalamus_ctr,brainstem_ctr,cerebellum_ctr,Watershed_ctr,Hemorrhagic_infarct_ctr,cortical_ACA_ctl,cortical_MCA_ctl,subcortical_ACA_ctl,subcortical_MCA_ctl,PCA_cortex_ctl,thalamus_ctl,brainstem_ctl,cerebellum_ctl,Watershed_ctl,Hemorrhagic_infarct_ctl,cortical_CT,subcortical_CT,circulation_CT,CT_find,watershed_CT,Hemorrhagic_infarct_CT,CT_left,CT_right,NIHS_1a_in,NIHS_1b_in,NIHS_1c_in,NIHS_2_in,NIHS_3_in,NIHS_4_in,NIHS_5aL_in,NIHS_5bR_in,NIHS_6aL_in,NIHS_6bR_in,NIHS_7_in,NIHS_8_in,NIHS_9_in,NIHS_10_in,NIHS_11_in
0,35.0,75.0,44.0,6.0,52.0,156.0,6.0,82.0,179.0,4.0,607.0,84.0,7.0,42.0,42.0,9.0,32.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,1.0,2.0,4.0,3.0,1.0,3.0,0.0,2.0,0.0,0.0,0.0
1,36.0,71.0,44.0,7.0,55.0,209.0,8.0,81.0,175.0,4.0,665.0,80.0,8.0,41.0,40.0,6.0,28.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,0.0,3.0,4.0,4.0,1.0,4.0,0.0,2.0,0.0,0.0,0.0
2,35.0,58.0,46.0,8.0,56.0,174.0,4.0,88.0,184.0,4.0,720.0,89.0,8.0,44.0,30.0,4.0,26.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,0.0,3.0,4.0,4.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0
3,36.0,96.0,40.0,8.0,53.0,250.0,9.0,80.0,167.0,3.0,557.0,76.0,7.0,41.0,43.0,12.0,37.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,2.0,1.0,2.0,3.0,3.0,1.0,3.0,1.0,2.0,0.0,0.0,0.0
4,36.0,84.0,44.0,7.0,55.0,219.0,8.0,83.0,186.0,4.0,676.0,84.0,8.0,42.0,37.0,6.0,28.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,0.0,3.0,4.0,4.0,1.0,4.0,1.0,2.0,0.0,0.0,0.0
5,35.0,71.0,46.0,6.0,55.0,121.0,7.0,88.0,189.0,4.0,708.0,89.0,7.0,43.0,34.0,6.0,28.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,0.0,3.0,4.0,4.0,1.0,4.0,0.0,2.0,0.0,0.0,0.0
6,36.0,97.0,39.0,9.0,48.0,309.0,11.0,71.0,151.0,3.0,520.0,71.0,7.0,38.0,51.0,10.0,48.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,2.0,1.0,2.0,3.0,3.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0
7,35.0,83.0,42.0,8.0,48.0,293.0,11.0,82.0,159.0,3.0,582.0,75.0,7.0,41.0,43.0,9.0,35.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,1.0,2.0,4.0,3.0,1.0,3.0,0.0,2.0,1.0,0.0,0.0
8,36.0,87.0,42.0,8.0,53.0,233.0,6.0,75.0,168.0,3.0,664.0,82.0,7.0,41.0,38.0,6.0,34.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,0.0,2.0,4.0,3.0,1.0,3.0,0.0,2.0,1.0,0.0,0.0
9,35.0,72.0,44.0,8.0,55.0,158.0,5.0,85.0,164.0,4.0,611.0,87.0,7.0,43.0,39.0,5.0,30.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,0.0,2.0,4.0,4.0,1.0,3.0,0.0,2.0,0.0,1.0,0.0


In [17]:
output_dataset.to_csv("../dataset/output_dataset/cWGAN_1d_2_models.csv",encoding='utf_8_sig')